In [4]:
# 1. Gerekli kütüphanelerin yüklenmesi
!pip install faiss-cpu numpy pandas tqdm
!pip install openai==0.28

# 2. Gerekli kütüphanelerin import edilmesi
import os
import json
import numpy as np
from tqdm import tqdm
import faiss
import openai

# 3. OpenAI API anahtarını ayarla
openai.api_key = ""

# 4. Veri dosyalarının yüklenmesi
train_file_path = "/content/sample_data/final_train_data_v2.json"
dev_file_path = "/content/sample_data/final_dev_data_v2.json"

# Dosyaların mevcut olup olmadığını kontrol edelim
assert os.path.exists(train_file_path), "Train dosyası bulunamadı!"
assert os.path.exists(dev_file_path), "Dev dosyası bulunamadı!"

# 5. JSON dosyasını yükleyelim ve inceleyelim
with open(train_file_path, "r", encoding="utf-8") as train_file:
    train_data = json.load(train_file)

with open(dev_file_path, "r", encoding="utf-8") as dev_file:
    dev_data = json.load(dev_file)

# 6. Veri yapısını inceleme
print(f"Training veri seti versiyonu: {train_data['version']}")
print(f"Development veri seti versiyonu: {dev_data['version']}")
print(f"Training veri setinde toplam {len(train_data['data'])} başlık mevcut.")
print(f"Development veri setinde toplam {len(dev_data['data'])} başlık mevcut.")


Training veri seti versiyonu: v0.2
Development veri seti versiyonu: v0.2
Training veri setinde toplam 756 başlık mevcut.
Development veri setinde toplam 85 başlık mevcut.


In [6]:
from tqdm import tqdm

# 7. Her paragraf için embedding oluşturma
def generate_embedding(text):
    """OpenAI API ile metni embedding'e çevirir."""
    response = openai.Embedding.create(
        model="text-embedding-ada-002",  # Uygun fiyatlı embedding modeli
        input=[text]  # input parametresi bir liste olmalı
    )
    return response['data'][0]['embedding']  # İlk embedding'i döndür

# 8. Eğitim verilerini embedding'e çevirme
train_embeddings = []  # Bu listeye embedding'ler ekleyeceğiz

print("Embedding oluşturuluyor...")
for topic in tqdm(train_data['data'], desc="Training Data"):
    for paragraph in topic['paragraphs']:
        embedding = generate_embedding(paragraph['context'])  # Paragrafın embedding'ini al
        train_embeddings.append(embedding)

# 9. Embedding'leri NumPy formatında saklayalım
train_embeddings_array = np.array(train_embeddings)
np.save("/content/sample_data/train_embeddings.npy", train_embeddings_array)
print("Embedding işlemi tamamlandı!")


Embedding oluşturuluyor...


Training Data: 100%|██████████| 756/756 [07:45<00:00,  1.62it/s]

Embedding işlemi tamamlandı!


In [7]:
# 10. FAISS dizini oluşturma
dimension = train_embeddings_array.shape[1]  # Embedding boyutu
index = faiss.IndexFlatL2(dimension)  # L2 normuna göre arama
index.add(train_embeddings_array)  # Embedding'leri FAISS'e ekle

print(f"FAISS dizinine {index.ntotal} embedding eklendi.")

# FAISS dizinini saklayalım
faiss.write_index(index, "/content/sample_data/faiss_index.bin")
print("FAISS dizini kaydedildi!")



FAISS dizinine 2400 embedding eklendi.
FAISS dizini kaydedildi!


In [8]:
# Sorgu için embedding oluşturma ve FAISS kullanarak en yakın belgeleri bulma
def retrieve_documents(query, k=5):
    """Kullanıcı sorgusu için en yakın k belgeyi döndürür."""
    query_embedding = np.array([generate_embedding(query)])
    distances, indices = index.search(query_embedding, k)
    return indices[0], distances[0]

# Örnek bir sorgu
query = "Osman Bey ne zaman doğmuştur?"
indices, distances = retrieve_documents(query)

print("Sorguya uygun bulunan belgeler:")
for i, idx in enumerate(indices):
    print(f"{i+1}. Belge ID: {idx}, Mesafe: {distances[i]}")


Sorguya uygun bulunan belgeler:
1. Belge ID: 70, Mesafe: 0.20507335662841797
2. Belge ID: 51, Mesafe: 0.22441300749778748
3. Belge ID: 231, Mesafe: 0.23101171851158142
4. Belge ID: 48, Mesafe: 0.23851671814918518
5. Belge ID: 30, Mesafe: 0.24012857675552368


In [40]:
def generate_fid_response(query, indices):
    """Fusion-in-Decoder ile bağlamlı yanıt oluşturur."""
    # Geçerli indeksleri filtreleme
    valid_indices = [idx for idx in indices if 0 <= idx < len(train_data['data'])]
    if not valid_indices:  # Eğer geçerli belge yoksa
        return "Belge bulunamadı."

    # Alınan belgeleri birleştirme
    context = "\n".join([train_data['data'][idx]['paragraphs'][0]['context'] for idx in valid_indices])

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Lütfen yalnızca tek veya iki kelimelik yanıtlar ver. Örneğin: 'Osman' gibi."},
        {"role": "user", "content": f"Kontekst: {context}\nSoru: {query}"}
    ]
)

    return response['choices'][0]['message']['content']


In [64]:
def generate_fid_response_long(query, indices):
    """Fusion-in-Decoder ile bağlamlı yanıt oluşturur."""
    # Geçerli indeksleri filtreleme
    valid_indices = [idx for idx in indices if 0 <= idx < len(train_data['data'])]
    if not valid_indices:  # Eğer geçerli belge yoksa
        return "Belge bulunamadı."

    # Alınan belgeleri birleştirme
    context = "\n".join([train_data['data'][idx]['paragraphs'][0]['context'] for idx in valid_indices])

    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Lütfen doğru ve tam yanıtlar ver."},
        {"role": "user", "content": f"Kontekst: {context}\nSoru: {query}"}
    ]
)

    return response['choices'][0]['message']['content']


In [41]:
# Örnek bir test sorusu
query = "Osman Bey nerede doğmuştur?"

# Retriever'dan bağlamı al
indices, distances = retrieve_documents(query)

# Bağlamdan yanıt oluştur
response = generate_fid_response(query, indices)

# Sonucu yazdır
print("Soru:", query)
print("Yanıt:", response)


Soru: Osman Bey nerede doğmuştur?
Yanıt: Bilecik


In [42]:
def normalize_text(text):
    import string
    text = text.lower().strip()
    return text.translate(str.maketrans('', '', string.punctuation))


In [48]:
# Test veri setinden soruları alalım
questions = [q['question'] for data in dev_data['data'] for q in data['paragraphs'][0]['qas']]
references = [q['answers'][0]['text'] for data in dev_data['data'] for q in data['paragraphs'][0]['qas']]

print(f"Test veri setinden {len(questions)} soru alındı.")

# Tahminleri üretelim
predictions = []

print("Tahminler üretiliyor...")
for question in tqdm(questions, desc="Processing Questions"):
    indices, _ = retrieve_documents(question)  # Retriever
    response = generate_fid_response(question, indices)  # Generator
    predictions.append(response)

print("Tüm tahminler üretildi.")


Test veri setinden 539 soru alındı.
Tahminler üretiliyor...


Processing Questions: 100%|██████████| 539/539 [04:45<00:00,  1.89it/s]

Tüm tahminler üretildi.


In [49]:
# Normalize edilmiş tahminler ve referanslar
normalized_predictions = [normalize_text(pred) for pred in predictions]
normalized_references = [normalize_text(ref) for ref in references]


In [50]:
def calculate_em_and_f1(predictions, references):
    exact_match = 0
    total_f1 = 0

    for pred, ref in zip(predictions, references):
        # Exact Match
        if pred.strip() == ref.strip():
            exact_match += 1

        # F1 Score
        pred_tokens = set(pred.split())
        ref_tokens = set(ref.split())
        common_tokens = pred_tokens & ref_tokens
        precision = len(common_tokens) / len(pred_tokens) if pred_tokens else 0
        recall = len(common_tokens) / len(ref_tokens) if ref_tokens else 0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        total_f1 += f1

    exact_match_score = exact_match / len(references) * 100
    f1_score_average = total_f1 / len(references) * 100
    return exact_match_score, f1_score_average

# EM ve F1 Score hesaplama
em_score, f1_score = calculate_em_and_f1(normalized_predictions, normalized_references)
print(f"Exact Match (EM): {em_score:.2f}%")
print(f"F1 Score: {f1_score:.2f}%")


Exact Match (EM): 3.90%
F1 Score: 8.07%


In [20]:
!pip install rouge-score nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c5fe5debfeccdb472e772c9c489b8ba0864deb529dd83e042f5d3a037ce21e05
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [51]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

from rouge_score import rouge_scorer
from rouge_score import rouge_scorer

def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge_scores["rouge1"].append(scores["rouge1"].fmeasure)
        rouge_scores["rouge2"].append(scores["rouge2"].fmeasure)
        rouge_scores["rougeL"].append(scores["rougeL"].fmeasure)

    # Ortalama ROUGE skorlarını hesapla
    avg_rouge1 = sum(rouge_scores["rouge1"]) / len(rouge_scores["rouge1"]) * 100
    avg_rouge2 = sum(rouge_scores["rouge2"]) / len(rouge_scores["rouge2"]) * 100
    avg_rougeL = sum(rouge_scores["rougeL"]) / len(rouge_scores["rougeL"]) * 100

    return avg_rouge1, avg_rouge2, avg_rougeL

# ROUGE metriklerini hesapla
rouge1, rouge2, rougeL = calculate_rouge(normalized_predictions, normalized_references)
print(f"ROUGE-1: {rouge1:.2f}%")
print(f"ROUGE-2: {rouge2:.2f}%")
print(f"ROUGE-L: {rougeL:.2f}%")
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(predictions, references):
    """BLEU skorlarını hesaplar."""
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = [ref.split()]  # BLEU için doğru yanıt bir liste içinde olmalı
        bleu_scores.append(sentence_bleu(ref_tokens, pred_tokens, weights=(1.0, 0.0, 0.0, 0.0)))  # BLEU-1
    return sum(bleu_scores) / len(bleu_scores) * 100

# BLEU metriklerini hesapla
bleu_score = calculate_bleu(normalized_predictions, normalized_references)
print(f"BLEU: {bleu_score:.2f}%")


ROUGE-1: 10.19%
ROUGE-2: 3.54%
ROUGE-L: 10.15%
BLEU: 6.78%


In [52]:
def calculate_recall_at_k(references, retriever_results, train_data, k=5):
    """Retriever'ın doğru bağlamı döndürme oranını hesaplar."""
    relevant = 0

    for ref, results in zip(references, retriever_results):
        # Retriever'ın döndürdüğü bağlamları al
        contexts = [
            train_data['data'][idx]['paragraphs'][0]['context']
            for idx in results[:k] if 0 <= idx < len(train_data['data'])
        ]
        # Bağlamların doğru yanıtı içerip içermediğini kontrol et
        if any(ref.lower() in context.lower() for context in contexts):
            relevant += 1

    recall = relevant / len(references) * 100
    return recall

# Retriever sonuçlarını al
retriever_results = [retrieve_documents(question)[0] for question in questions]

# Recall@5 hesapla
recall_at_5 = calculate_recall_at_k(normalized_references, retriever_results, train_data, k=5)
print(f"Recall@5: {recall_at_5:.2f}%")


Recall@5: 1.67%


In [53]:
import pandas as pd

data = {
    "Metric": ["Exact Match (EM)", "F1 Score", "ROUGE-1", "ROUGE-2", "ROUGE-L", "BLEU", "Recall@5"],
    "Value": [em_score, f1_score, rouge1, rouge2, rougeL, bleu_score, recall_at_5]
}

df = pd.DataFrame(data)
print(df)


             Metric      Value
0  Exact Match (EM)   3.896104
1          F1 Score   8.068961
2           ROUGE-1  10.189030
3           ROUGE-2   3.539408
4           ROUGE-L  10.146770
5              BLEU   6.783776
6          Recall@5   1.669759


In [69]:
# Örnek bir test sorusu
query = "Ertuğrul Bey kaç yaşında vefat etmiştir?"

# Retriever'dan bağlamı al
indices, distances = retrieve_documents(query)

# Bağlamdan yanıt oluştur
response = generate_fid_response_long(query, indices)

# Sonucu yazdır
print("Soru:", query)
print("Yanıt:", response)


Soru: Ertuğrul Bey kaç yaşında vefat etmiştir?
Yanıt: Belge bulunamadı.
